# 🚑 Hackathon Routing Notebook (ORS-only)

This notebook computes a **6×6 cost matrix** using **OpenRouteService (ORS)**, solves the routes with:
- **Exact 3+2 split (brute force)**,
- **Heuristic (split + nearest-insertion + 2-opt)**,
and draws **snapped markers** + **route polylines** on interactive maps.

**Tip:** Run cells from top to bottom. Set your ORS API key in the next section.


## 🧱 Block 0 — minimal installs (Colab)


In [ ]:
!pip -q install openrouteservice folium pandas

## 🧱 Block 1 — load JSON + show raw points


In [ ]:
import json, pandas as pd, folium

# Path to your JSON (update if needed)
JSON_PATH = "/content/OptimizationProblemData.json"

with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

hospital = data["locations"]["hospital"]["coordinates"]
patients = data["locations"]["patients"]

labels  = ["H"] + [p["id"] for p in patients]
coords  = [(hospital["latitude"], hospital["longitude"])] + [
    (p["coordinates"]["latitude"], p["coordinates"]["longitude"]) for p in patients
]

print("=== Loaded points ===")
for lab, (lat, lon) in zip(labels, coords):
    print(f"{lab:>5}: lat={lat:.6f}, lon={lon:.6f}")

# quick raw map
m_raw = folium.Map(location=[coords[0][0], coords[0][1]], zoom_start=13, tiles="cartodbpositron")
for i, (lat, lon) in enumerate(coords):
    color = "red" if i == 0 else "blue"
    icon  = "hospital-o" if i == 0 else "user"
    folium.Marker([lat, lon], tooltip=labels[i],
                  icon=folium.Icon(color=color, icon=icon, prefix="fa")).add_to(m_raw)
m_raw

=== Loaded points ===
    H: lat=29.995127, lon=31.684628
   DT: lat=30.000418, lon=31.739608
   GR: lat=30.011344, lon=31.747827
   R2: lat=30.030388, lon=31.669231
 R3_2: lat=30.030941, lon=31.688371
   IT: lat=30.012856, lon=31.693812


## 🧱 Block 2 — ORS init (paste your API key)


In [ ]:
import os, openrouteservice as ors

# 🔑 Paste your ORS key here OR set it via: os.environ["ORS_API_KEY"] = "..."
os.environ["ORS_API_KEY"] = os.getenv("ORS_API_KEY", "eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6IjNkYmExZTllYTBhYzRkODNhYmIwMjJkZTQ2ZTk3ZmJjIiwiaCI6Im11cm11cjY0In0=")
client = ors.Client(key=os.environ["ORS_API_KEY"])  # will raise if key invalid
print("ORS client ready.")

ORS client ready.


## 🧱 Block 3 — ORS cost matrix + ORS-snapped markers


In [ ]:
import pandas as pd

# ORS expects (lon, lat)
locs_lonlat = [(lon, lat) for (lat, lon) in coords]
n = len(locs_lonlat)
idx = list(range(n))

# Matrix request (distance in meters) + let ORS snap your inputs to its graph
try:
    res = client.matrix(
        locations=locs_lonlat,
        profile="driving-car",
        metrics=["distance"],
        sources=idx,
        destinations=idx,
        resolve_locations=True
    )
except Exception:
    # some client versions expose distance_matrix instead of matrix
    res = client.distance_matrix(
        locations=locs_lonlat,
        profile="driving-car",
        metrics=["distance"],
        sources=idx,
        destinations=idx,
        resolve_locations=True
    )

# Build C (km) for your solver
Dmat = res.get("distances")
if Dmat is None:
    raise RuntimeError("ORS response missing 'distances'. Check API key/quota.")

C = [[0.0]*n for _ in range(n)]
for i in range(n):
    for j in range(n):
        if i != j and Dmat[i][j] is not None:
            C[i][j] = float(Dmat[i][j]) / 1000.0  # m → km

print("\n=== Cost Matrix (km, ORS driving-car) ===")
display(pd.DataFrame(C, index=labels, columns=labels).round(3))

# Keep ORS-snapped waypoints to align all visuals (returned in same order as your inputs)
# Each element is {"location": [lon,lat], ...}
ors_snapped_lonlat = [tuple(src["location"]) for src in res["sources"]]
ors_snapped_latlon = [(lat, lon) for (lon, lat) in ors_snapped_lonlat]

print("\nSnapped waypoints (lat, lon):")
for lab, (lat, lon) in zip(labels, ors_snapped_latlon):
    print(f"  {lab}: lat={lat:.6f}, lon={lon:.6f}")



=== Cost Matrix (km, ORS driving-car) ===


,H,DT,GR,R2,R3_2,IT
H,0.000,8.596,11.466,9.415,10.836,9.488
DT,14.157,0.000,2.362,10.918,9.245,9.434
GR,15.039,7.747,0.000,11.800,10.128,10.476
R2,11.821,17.764,15.640,0.000,11.576,11.520
R3_2,7.617,12.473,10.350,4.378,0.000,6.230
IT,9.236,9.401,12.271,7.321,8.742,0.000



Snapped waypoints (lat, lon):
  H: lat=29.995177, lon=31.685001
  DT: lat=30.007644, lon=31.739652
  GR: lat=30.011828, lon=31.747268
  R2: lat=30.030448, lon=31.669397
  R3_2: lat=30.031028, lon=31.687846
  IT: lat=30.012904, lon=31.693403


In [ ]:
# ORS-snapped waypoints (same order as inputs)
ors_snapped_lonlat = [tuple(src["location"]) for src in res["sources"]]      # (lon,lat)
ors_snapped_latlon = [(lat, lon) for (lon, lat) in ors_snapped_lonlat]          # (lat,lon)

print("\nSnapped waypoints (lat, lon):")
for lab, (lat, lon) in zip(labels, ors_snapped_latlon):
    print(f"  {lab}: lat={lat:.6f}, lon={lon:.6f}")

# Map: raw (blue) vs snapped (green)
m_snap = folium.Map(location=[coords[0][0], coords[0][1]], zoom_start=13, tiles="cartodbpositron")
for i,(lat,lon) in enumerate(coords):
    folium.Marker([lat, lon], tooltip=f"{labels[i]} (raw)",
                  icon=folium.Icon(color="blue", icon="user", prefix="fa")).add_to(m_snap)
for i,(lat,lon) in enumerate(ors_snapped_latlon):
    folium.CircleMarker([lat, lon], radius=6, color="green", fill=True, fill_opacity=1.0,
                        tooltip=f"{labels[i]} (ORS snapped)").add_to(m_snap)
for i in range(n):
    folium.PolyLine([[coords[i][0], coords[i][1]], [ors_snapped_latlon[i][0], ors_snapped_latlon[i][1]]],
                    color="#888888", weight=2, opacity=0.6).add_to(m_snap)
m_snap


Snapped waypoints (lat, lon):
  H: lat=29.995177, lon=31.685001
  DT: lat=30.007644, lon=31.739652
  GR: lat=30.011828, lon=31.747268
  R2: lat=30.030448, lon=31.669397
  R3_2: lat=30.031028, lon=31.687846
  IT: lat=30.012904, lon=31.693403


## 🧱 Block 4 — Exact solver (3+2 brute force)


In [ ]:
import itertools as it

def trip_cost(order, C):
    if not order: return 0.0
    cost = C[0][order[0]]
    for a,b in zip(order, order[1:]): cost += C[a][b]
    cost += C[order[-1]][0]
    return cost

def fmt(order):
    return "H → " + " → ".join(labels[i] for i in order) + " → H"

# exact: enumerate all 3+2 partitions & permutations
P = list(range(1, len(labels)))   # patient indices (exclude H=0)
cands = []
for trip1_set in it.combinations(P, 3):
    trip2_set = tuple(sorted(set(P) - set(trip1_set)))
    best1 = min(((trip_cost(list(p), C), list(p)) for p in it.permutations(trip1_set)), key=lambda x:x[0])
    best2 = min(((trip_cost(list(p), C), list(p)) for p in it.permutations(trip2_set)), key=lambda x:x[0])
    total = best1[0] + best2[0]
    cands.append({"t1":best1[1], "c1":best1[0], "t2":best2[1], "c2":best2[0], "tot":total})
cands.sort(key=lambda d: d["tot"])
best_exact = cands[0]

print("\n=== Exact (3,2) ===")
print(f"Trip 1: {fmt(best_exact['t1'])} | {best_exact['c1']:.2f} km")
print(f"Trip 2: {fmt(best_exact['t2'])} | {best_exact['c2']:.2f} km")
print(f"TOTAL : {best_exact['tot']:.2f} km")


=== Exact (3,2) ===
Trip 1: H → DT → GR → R3_2 → H | 28.70 km
Trip 2: H → IT → R2 → H | 28.63 km
TOTAL : 57.33 km


## 🧱 Block 5 — Heuristic (split + nearest-insertion + 2-opt)


In [ ]:
import random
random.seed(42)

def nearest_insertion(order_list):
    if not order_list: return []
    first = min(order_list, key=lambda i: C[0][i])
    tour = [first]
    remaining = [i for i in order_list if i != first]
    while remaining:
        nxt = min(remaining, key=lambda j: min(C[i][j] for i in tour))
        best_pos, best_inc = 0, float("inf")
        for k in range(len(tour)+1):
            prev = 0 if k == 0 else tour[k-1]
            nxt_after = 0 if k == len(tour) else tour[k]
            inc = C[prev][nxt] + C[nxt][nxt_after] - C[prev][nxt_after]
            if inc < best_inc:
                best_inc, best_pos = inc, k
        tour.insert(best_pos, nxt)
        remaining.remove(nxt)
    return tour

def two_opt(tour):
    improved = True
    best = tour[:]
    def ring_cost(t): return trip_cost(t, C)
    while improved:
        improved = False
        for i in range(len(best)-1):
            for j in range(i+1, len(best)):
                new_t = best[:i] + best[i:j+1][::-1] + best[j+1:]
                if ring_cost(new_t) + 1e-9 < ring_cost(best):
                    best = new_t
                    improved = True
    return best

def heuristic_partition(capacity=3, trials=64):
    P = list(range(1, len(labels)))
    best = None
    best_total = float("inf")
    for _ in range(trials):
        random.shuffle(P)
        t1 = P[:capacity]
        t2 = P[capacity:]
        r1 = two_opt(nearest_insertion(t1))
        r2 = two_opt(nearest_insertion(t2))
        total = trip_cost(r1, C) + trip_cost(r2, C)
        if total < best_total:
            best_total = total
            best = (r1, r2)
    return best, best_total

best_h, best_h_cost = heuristic_partition(capacity=3, trials=64)

print("\n=== Heuristic (split + NI + 2-opt) ===")
print(f"Trip 1: {fmt(best_h[0])} | {trip_cost(best_h[0], C):.2f} km")
print(f"Trip 2: {fmt(best_h[1])} | {trip_cost(best_h[1], C):.2f} km")
print(f"TOTAL : {best_h_cost:.2f} km")


=== Heuristic (split + NI + 2-opt) ===
Trip 1: H → DT → GR → R3_2 → H | 28.70 km
Trip 2: H → IT → R2 → H | 28.63 km
TOTAL : 57.33 km


## 🧱 Block 6 — Draw ORS routes (Exact & Heuristic) with layers


In [ ]:
def ors_route_line(order):
    seq = [0] + list(order) + [0]
    seq_coords = [ors_snapped_lonlat[i] for i in seq]  # (lon,lat)
    geo = client.directions(
        coordinates=seq_coords,
        profile="driving-car",
        format="geojson",
        optimize_waypoints=False
    )
    line = geo["features"][0]["geometry"]["coordinates"]  # [lon,lat]
    return [(lat, lon) for (lon, lat) in line]

ex1 = ors_route_line(best_exact["t1"])  # exact trip 1
ex2 = ors_route_line(best_exact["t2"])  # exact trip 2
h1  = ors_route_line(best_h[0])          # heuristic trip 1
h2  = ors_route_line(best_h[1])          # heuristic trip 2

m_routes = folium.Map(location=[ors_snapped_latlon[0][0], ors_snapped_latlon[0][1]],
                      zoom_start=13, tiles="cartodbpositron")

# raw (blue) + snapped (green) markers
for i,(lat,lon) in enumerate(coords):
    folium.Marker([lat, lon], tooltip=f"{labels[i]} (raw)",
                  icon=folium.Icon(color="blue", icon="user", prefix="fa")).add_to(m_routes)
for i,(lat,lon) in enumerate(ors_snapped_latlon):
    folium.CircleMarker([lat, lon], radius=6, color="green", fill=True, fill_opacity=1.0,
                        tooltip=f"{labels[i]} (snapped)").add_to(m_routes)

fg_exact = folium.FeatureGroup(name="Exact (ORS)")
folium.PolyLine(ex1, weight=6, opacity=0.9, color="red",
                tooltip=f"Exact Trip 1: H → " + " → ".join(labels[i] for i in best_exact['t1']) + " → H").add_to(fg_exact)
folium.PolyLine(ex2, weight=6, opacity=0.9, color="green",
                tooltip=f"Exact Trip 2: H → " + " → ".join(labels[i] for i in best_exact['t2']) + " → H").add_to(fg_exact)
fg_exact.add_to(m_routes)

fg_heur = folium.FeatureGroup(name="Heuristic (ORS)")
folium.PolyLine(h1, weight=5, opacity=0.8, color="blue",
                tooltip=f"Heuristic Trip 1: H → " + " → ".join(labels[i] for i in best_h[0]) + " → H").add_to(fg_heur)
folium.PolyLine(h2, weight=5, opacity=0.8, color="purple",
                tooltip=f"Heuristic Trip 2: H → " + " → ".join(labels[i] for i in best_h[1]) + " → H").add_to(fg_heur)
fg_heur.add_to(m_routes)

folium.LayerControl(collapsed=False).add_to(m_routes)
m_routes

## 🧱 (Optional) Block 7 — Save maps to HTML (for submission)


In [ ]:
m_raw.save("00_raw_points.html")
m_snap.save("01_raw_vs_snapped.html")
m_routes.save("02_routes_exact_vs_heuristic.html")
print("Saved:", "00_raw_points.html, 01_raw_vs_snapped.html, 02_routes_exact_vs_heuristic.html")

Saved: 00_raw_points.html, 01_raw_vs_snapped.html, 02_routes_exact_vs_heuristic.html


In [ ]:
# --- Block 1: Build directed edges e_ij and distances d_ij, plus bitstring helpers ---

import itertools as it
import numpy as np
import pandas as pd

# sanity
assert labels[0] == "H", "labels[0] must be hospital 'H'"
N = len(labels)

# 1) edge list E = all directed pairs i!=j
E = [(i, j) for i in range(N) for j in range(N) if i != j]

# 2) distances d_ij from your cost matrix C (km)
#    d_ij is just C[i][j]; wrap into a dict for convenience
d = {(i, j): float(C[i][j]) for (i, j) in E}

# 3) indexing for bitstrings over edges
edge_to_idx = {e: k for k, e in enumerate(E)}
idx_to_edge = {k: e for e, k in edge_to_idx.items()}
num_edges = len(E)

print(f"Directed edges: {num_edges} bits")
print("First 10 edges:", E[:300])

# 4) helper: encode a pair of routes as an edge-bitstring
#    routes are sequences of node indices *excluding* the hospital (0)
#    each route is executed as H->...->H
def routes_to_edge_bits(route1, route2):
    """
    route1: list[int] e.g., [i,j,k]  (3 customers)
    route2: list[int] e.g., [m,n]    (2 customers)
    returns: np.array of shape (num_edges,), 0/1
    """
    bits = np.zeros(num_edges, dtype=int)

    def activate_path(order):
        seq = [0] + order + [0]
        for a, b in zip(seq, seq[1:]):
            if a != b:
                k = edge_to_idx[(a, b)]
                bits[k] = 1

    activate_path(route1)
    activate_path(route2)
    return bits

# 5) pretty-printers
def fmt_route(order):
    return "H → " + " → ".join(labels[i] for i in order) + " → H"

def bitstring_to_edges(bits):
    return [idx_to_edge[k] for k, v in enumerate(bits) if v == 1]

# 6) quick test (use your current best routes if you have them)
# (example placeholders; replace with your actual best orders)
# r1 = [2,1,4]   # e.g., labels indices for a 3-stop route
# r2 = [5,3]     # e.g., labels indices for a 2-stop route
# bits = routes_to_edge_bits(r1, r2)
# print(fmt_route(r1), "| edges on:", bitstring_to_edges(bits)[:6], "...")


Directed edges: 30 bits
First 10 edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 0), (1, 2), (1, 3), (1, 4), (1, 5), (2, 0), (2, 1), (2, 3), (2, 4), (2, 5), (3, 0), (3, 1), (3, 2), (3, 4), (3, 5), (4, 0), (4, 1), (4, 2), (4, 3), (4, 5), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4)]


In [ ]:
# --- Block 2: Build a plain position-based QUBO (quadratic) from C ---

from collections import defaultdict

# positions per route
k1, k2 = 3, 2
patients = list(range(1, N))  # patients are 1..N-1

# variable indexing: map (route, i, p) -> qindex
# route r in {1,2}; i in patients; p in {1..k_r}
var_to_idx = {}
idx_to_var = {}
q = 0
for r, K in [(1, k1), (2, k2)]:
    for i in patients:
        for p in range(1, K+1):
            var_to_idx[(r, i, p)] = q
            idx_to_var[q] = (r, i, p)
            q += 1

num_vars = q
print("QUBO vars (position one-hot):", num_vars)

# helpers for QUBO assembly
Q = defaultdict(float)  # store upper-triangular (i<=j) entries
const = 0.0

def add_lin(u, c):
    Q[(u, u)] += c

def add_quad(u, v, c):
    if u == v:
        Q[(u, u)] += c
    else:
        a, b = (u, v) if u < v else (v, u)
        Q[(a, b)] += c

# penalty strength
edge_max = max(C[i][j] for i in range(N) for j in range(N) if i != j)
A = 10.0 * edge_max  # you can tune; must dominate a single edge cost

# (I) Objective terms: depot->first, between positions, last->depot
# Route 1
for i in patients:
    add_lin(var_to_idx[(1, i, 1)], C[0][i])      # H -> first
    add_lin(var_to_idx[(1, i, k1)], C[i][0])     # last -> H
for p in range(1, k1):  # between positions p and p+1
    for i in patients:
        for j in patients:
            add_quad(var_to_idx[(1, i, p)], var_to_idx[(1, j, p+1)], C[i][j])

# Route 2
for i in patients:
    add_lin(var_to_idx[(2, i, 1)], C[0][i])      # H -> first
    add_lin(var_to_idx[(2, i, k2)], C[i][0])     # last -> H
for p in range(1, k2):
    for i in patients:
        for j in patients:
            add_quad(var_to_idx[(2, i, p)], var_to_idx[(2, j, p+1)], C[i][j])

# (II) Constraints: exactly one patient at each position
def add_exactly_one(vars_group, W):
    global const
    # (sum z - 1)^2 = -sum z + 2 sum_{u<v} z_u z_v + 1
    for u in vars_group:
        add_lin(u, -W)
    for a in range(len(vars_group)):
        for b in range(a+1, len(vars_group)):
            add_quad(vars_group[a], vars_group[b], 2.0*W)
    const += W

# positions in each route
for p in range(1, k1+1):
    grp = [var_to_idx[(1, i, p)] for i in patients]
    add_exactly_one(grp, A)

for p in range(1, k2+1):
    grp = [var_to_idx[(2, i, p)] for i in patients]
    add_exactly_one(grp, A)

# (III) Constraints: each patient used exactly once across both routes
for i in patients:
    grp = [var_to_idx[(1, i, p)] for p in range(1, k1+1)] + \
          [var_to_idx[(2, i, p)] for p in range(1, k2+1)]
    add_exactly_one(grp, A)

# Convert Q (dict) to dense upper-tri matrix (if needed)
# For QAOA via Qiskit Optimization we’ll add terms directly, but keep this around if you want Q explicitly:
def qubo_dense(Qdict, nvars):
    Qm = np.zeros((nvars, nvars))
    for (i, j), v in Qdict.items():
        Qm[i, j] += v
        if i != j:
            Qm[j, i] += v
    return Qm

Q_dense = qubo_dense(Q, num_vars)
print("QUBO matrix shape:", Q_dense.shape, "| nonzeros:", int(np.count_nonzero(Q_dense)))


QUBO vars (position one-hot): 25
QUBO matrix shape: (25, 25) | nonzeros: 345


In [ ]:
# ===== PennyLane QAOA (explicit gates, wire-safe) =====
%pip -q install pennylane pennylane-lightning

import pennylane as qml
from pennylane import numpy as pnp
import numpy as np

# --- sanity: require Q_dense and const ---
if "Q_dense" not in globals() or "const" not in globals():
    raise RuntimeError("Please define Q_dense (numpy array) and const (float) before running this cell.")

# ---- QUBO (0/1) -> Ising (Z in {-1,+1}) ----
def qubo_to_ising(Q, const=0.0):
    Q = np.array(Q, dtype=float)
    n = Q.shape[0]
    c = float(const)
    h = np.zeros(n, dtype=float)         # 1-local Z coeffs
    J = {}                                # 2-local ZZ coeffs, keys (i,j) with i<j

    # diagonal -> linear Z and constant
    for i in range(n):
        qii = Q[i, i]
        if qii != 0.0:
            c += 0.5 * qii
            h[i] += -0.5 * qii

    # off-diagonal -> ZZ + linear Z + constant
    for i in range(n):
        for j in range(i+1, n):
            qij = Q[i, j] + Q[j, i]  # tolerate asymmetry
            if qij != 0.0:
                c += 0.25 * qij
                h[i] += -0.25 * qij
                h[j] += -0.25 * qij
                J[(i, j)] = J.get((i, j), 0.0) + 0.25 * qij
    return c, h, J

const_I, h_vec, J_map = qubo_to_ising(Q_dense, const)

# ---- wires and pair indexing ----
n_qubits = len(h_vec)
pairs = list(J_map.items())                       # [((i,j), Jij), ...]
pairs_ij = [p[0] for p in pairs]                  # [(i,j), ...]
pairs_w = pnp.array([p[1] for p in pairs])        # weights (Jij) as array
h_vec = pnp.array(h_vec, dtype=float)             # convert to PL array

print(f"[QAOA] n_qubits={n_qubits}, |h|={len(h_vec)}, |J|={len(pairs)}")

# ---- Devices ----
try:
    dev = qml.device("lightning.qubit", wires=n_qubits)  # analytic for expectation
    print("[QAOA] Using lightning.qubit for expectations")
except Exception:
    dev = qml.device("default.qubit", wires=n_qubits)
    print("[QAOA] Using default.qubit for expectations")

try:
    dev_sample = qml.device("lightning.qubit", wires=n_qubits, shots=8192)
    print("[QAOA] Using lightning.qubit (shots=8192) for sampling")
except Exception:
    dev_sample = qml.device("default.qubit", wires=n_qubits, shots=8192)
    print("[QAOA] Using default.qubit (shots=8192) for sampling")

# ---- QAOA layers (explicit gates; no Hamiltonian objects) ----
def cost_layer_explicit(gamma):
    # 2-local ZZ terms
    for (i, j), Jij in pairs:
        if Jij != 0.0:
            qml.IsingZZ(2.0 * gamma * float(Jij), wires=[i, j])
    # 1-local Z terms
    for i in range(n_qubits):
        hi = float(h_vec[i])
        if hi != 0.0:
            qml.RZ(2.0 * gamma * hi, wires=i)

def mixer_layer_explicit(beta):
    for i in range(n_qubits):
        qml.RX(2.0 * beta, wires=i)

# ---- Expectation node returns vector of <Z_i> and <Z_i Z_j> we combine outside ----
@qml.qnode(dev)
def qaoa_measure(params):
    p = params.shape[0]
    # Start |+>^n
    for w in range(n_qubits):
        qml.Hadamard(wires=w)
    # Layers
    for s in range(p):
        gamma, beta = params[s, 0], params[s, 1]
        cost_layer_explicit(gamma)
        mixer_layer_explicit(beta)
    # Measure singles and pairs
    singles = [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    pairs_meas = [qml.expval(qml.PauliZ(i) @ qml.PauliZ(j)) for (i, j) in pairs_ij]
    return tuple(singles + pairs_meas)

def energy_from_measure(vec):
    """vec = [<Z0>,...,<Z_{n-1}>, <Z_i Z_j>...] in the same order we defined above."""
    singles = pnp.array(vec[:n_qubits], dtype=float)
    pairs_val = pnp.array(vec[n_qubits:], dtype=float)
    return pnp.dot(h_vec, singles) + pnp.dot(pairs_w, pairs_val) + const_I

def qaoa_energy(params):
    vals = qaoa_measure(params)
    return energy_from_measure(vals)

# ---- Sampler node for bitstrings ----
@qml.qnode(dev_sample)
def qaoa_sample(params):
    p = params.shape[0]
    for w in range(n_qubits):
        qml.Hadamard(wires=w)
    for s in range(p):
        gamma, beta = params[s, 0], params[s, 1]
        cost_layer_explicit(gamma)
        mixer_layer_explicit(beta)
    return qml.sample(wires=range(n_qubits))

# ---- Optimize parameters ----
p = 1  # increase to 2-3 if you have the time
params = pnp.array([[0.5, 0.5] for _ in range(p)], requires_grad=True)
opt = qml.AdamOptimizer(stepsize=0.1)

best_params = params.copy()
best_val = 1e99
steps = 150
for t in range(steps):
    params, val = opt.step_and_cost(qaoa_energy, params)
    if val < best_val:
        best_val = val
        best_params = params.copy()
    if (t+1) % 25 == 0 or t == 0:
        print(f"iter {t+1:3d} | ⟨H⟩ = {val:.6f}")

print(f"[QAOA] best ⟨H⟩ = {best_val:.6f}")

# ---- Sampling + bitstring ----
shots = qaoa_sample(best_params)
counts = {}
for s in shots:
    key = "".join(str(int(b)) for b in s)
    counts[key] = counts.get(key, 0) + 1
best_bitstr = max(counts, key=counts.get)
print("Best bitstring:", best_bitstr, "| count:", counts[best_bitstr])

x = np.array([int(b) for b in best_bitstr], dtype=int)

# ---- Optional: decode routes if your globals exist (var_to_idx, patients, k1, k2, C) ----
def _maybe(name):
    try: return globals()[name]
    except KeyError: return None

var_to_idx = _maybe("var_to_idx")
patients   = _maybe("patients")
k1         = _maybe("k1")
k2         = _maybe("k2")
C          = _maybe("C")

def decode_routes(x):
    r1, r2 = [], []
    if var_to_idx is None:
        return r1, r2
    if k1:
        for ppos in range(1, k1+1):
            picks = [(i, var_to_idx[(1, i, ppos)]) for i in patients if (1, i, ppos) in var_to_idx]
            if picks:
                r1.append(max(picks, key=lambda t: x[t[1]])[0])
    if k2:
        for ppos in range(1, k2+1):
            picks = [(i, var_to_idx[(2, i, ppos)]) for i in patients if (2, i, ppos) in var_to_idx]
            if picks:
                r2.append(max(picks, key=lambda t: x[t[1]])[0])
    return r1, r2

def route_cost(order):
    if C is None or not order: return 0.0
    cost = C[0][order[0]]
    for a, b in zip(order, order[1:]):
        cost += C[a][b]
    cost += C[order[-1]][0]
    return cost

if all(v is not None for v in [var_to_idx, patients, k1, k2, C]):
    r1, r2 = decode_routes(x)
    print("Route 1:", r1, "|", f"{route_cost(r1):.2f} km")
    print("Route 2:", r2, "|", f"{route_cost(r2):.2f} km")
    print("TOTAL  :", f"{route_cost(r1)+route_cost(r2):.2f} km")
else:
    print("(Skipping route decode — supply var_to_idx/patients/k1/k2/C to print routes.)")

[QAOA] n_qubits=25, |h|=25, |J|=160
[QAOA] Using lightning.qubit for expectations
[QAOA] Using lightning.qubit (shots=8192) for sampling


# 📝 Methodology & Evolution of Our Solution

## 1. Starting Point
- **Graph engine:** OSMnx to build a local road network + NetworkX Dijkstra for shortest paths.  
- **Snapping:** Projected hospital/patient coordinates to nearest nodes/edges in the OSMnx graph.  
- **Heuristic:** Randomized Nearest Neighbor (RNN) as the first attempt.  

---

## 2. Problems We Found
- **Snapping issue:** OSMnx sometimes snapped points incorrectly (e.g. “DT” outside road network, U-shaped detours).  
- **RNN weakness:** Sanity tests showed RNN performed poorly, even after tuning.  
- **Scalability:** Considered replacing Dijkstra with Tsinghua’s new SSSP algorithm (Rust POC exists), but it was not production-ready for our timeline.  

---

## 3. Shifts & Improvements
- **Snapping fix:** Adopted **OpenRouteService (ORS)** snapping → more accurate and consistent with routing engine.  
- **Heuristic upgrade:**  
  - Dropped RNN.  
  - Adopted a **split + nearest insertion + 2-opt** approach.  
  - Validated against brute-force (3,2 split) → matched exact optimum on test datasets.  
- **Exact solver:** For small N, brute force gave a **gold standard** to benchmark heuristics.  

---

## 4. Key Lessons
1. **Snapping is critical** — ORS solved the U-turn and “DT off-road” issues.  
2. **Always sanity check heuristics** — RNN looked fine on paper but was poor in practice.  
3. **Theory vs practice** — Tsinghua SSSP is exciting, but reliability and clarity win in hackathon settings.  
4. **Final pipeline:**  
   - Input JSON → ORS snapping & cost matrix  
   - Brute-force exact (small) + heuristic (scalable)  
   - ORS polylines visualization for clarity
